# Classify price movements based on candlesticks statistics

This reseach comes from [this](https://www.forexfactory.com/thread/post/14707863#post14707863) post on ForexFactory. 

### Step 1: gather data and create a rolling TF

In [1]:
# This allow jupiter to upload in real time externally modified code
%load_ext autoreload
%autoreload 2 

from x_CLASSES.download_data import DownloadData
import os
import pandas as pd
import numpy as np

In [2]:
start_date = "01-12-2023"
end_date = "24-12-2023"
timeframe = 'tick'

csv_file_path = f"{os.getcwd()}/x_DATA/{start_date}_{end_date}  {timeframe}.csv"


if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    df = DownloadData('GBP/USD', start_date,end_date,timeframe).getData()
    
    data_folder_path = f"{os.getcwd()}/x_DATA"
    if not os.path.exists(data_folder_path):
        os.makedirs(data_folder_path)

    df.to_csv(csv_file_path)

df

,Timestamp,Bid,Ask
0,2023-12-01 00:00:02.033000+00:00,1.26305,1.26316
1,2023-12-01 00:00:05.974000+00:00,1.26306,1.26316
2,2023-12-01 00:00:07.587000+00:00,1.26305,1.26316
3,2023-12-01 00:00:07.689000+00:00,1.26306,1.26316
4,2023-12-01 00:00:07.792000+00:00,1.26304,1.26317
...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26820
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26821
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26820
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26819


In [3]:
if "Ask" in df.columns:
    df = df.drop("Ask", axis=1)

df["Close"] = df["Bid"]
df["Open"] = df["Bid"].shift(1000)  # Shift the "Bid" values 1000 rows back
df["High"] = df["Bid"].rolling(window=1000).max()  # Calculate the rolling max over the last 1000 rows
df["Low"] = df["Bid"].rolling(window=1000).min()  # Calculate the rolling min over the last 1000 rows

df = df.dropna()
df

,Timestamp,Bid,Close,Open,High,Low
1000,2023-12-01 00:21:58.140000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298
1001,2023-12-01 00:22:00.557000+00:00,1.26377,1.26377,1.26306,1.26397,1.26298
1002,2023-12-01 00:22:00.912000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298
1003,2023-12-01 00:22:01.166000+00:00,1.26378,1.26378,1.26306,1.26397,1.26298
1004,2023-12-01 00:22:01.317000+00:00,1.26378,1.26378,1.26304,1.26397,1.26298
...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795


### Step 2: Directional bias

In [4]:
df['Open - Close'] = (df['Open'] - df['Close']).abs()
df['Upper Wick'] = (df['High'] - df[['Open', 'Close']].max(axis=1)).abs()
df['Lower Wick'] = (df[['Open', 'Close']].min(axis=1) - df['Low']).abs()

df['Bias'] = np.where(df['Open'] > df['Close'], 'Bearish', np.where(df['Close'] > df['Open'], 'Bullish', 'Doji'))

df


,Timestamp,Bid,Close,Open,High,Low,Open - Close,Upper Wick,Lower Wick,Bias
1000,2023-12-01 00:21:58.140000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298,0.00073,0.00019,0.00007,Bullish
1001,2023-12-01 00:22:00.557000+00:00,1.26377,1.26377,1.26306,1.26397,1.26298,0.00071,0.00020,0.00008,Bullish
1002,2023-12-01 00:22:00.912000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298,0.00073,0.00019,0.00007,Bullish
1003,2023-12-01 00:22:01.166000+00:00,1.26378,1.26378,1.26306,1.26397,1.26298,0.00072,0.00019,0.00008,Bullish
1004,2023-12-01 00:22:01.317000+00:00,1.26378,1.26378,1.26304,1.26397,1.26298,0.00074,0.00019,0.00006,Bullish
...,...,...,...,...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795,0.00024,0.00022,0.00015,Bearish
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795,0.00025,0.00023,0.00013,Bearish
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795,0.00027,0.00021,0.00013,Bearish
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795,0.00035,0.00016,0.00010,Bearish


### Step 3: Strength of directional bias
It can be:
- weak
- medium ("normal")
- strong

I'll calculate the average candlestick like: $\frac{\text{Average Bullish candlestick OC}+\text{Average Bearish candlestick OC}}{2}$. 
Then I'll calculate the standard deviation of the bullish candles OC and bearish candles OC and then: $\frac{\text{St.dev Bullish candlestick OC}+\text{St.dev Bearish candlestick OC}}{2}$. 
And lastly I'll create a boundary around the average, meaning that:
- If a bullish candle OC is $>=\mu+\sigma$ then it's strong. 
- If a bullish candle OC is $>=\mu$ and $<\mu+\sigma$ then it's medium ("normal").
- If a bullish candle OC is $<=\mu-\sigma$ then it's weak.

In [5]:

# Filter rows where Bias is "Bullish"
bullish_rows = df[df['Bias'] == 'Bullish']
bearish_rows = df[df['Bias'] == 'Bearish']

# average bullish and bearish candle OC
average_bullish_diff = bullish_rows['Open - Close'].mean()
average_bearish_diff = bearish_rows['Open - Close'].mean()
mu = ( average_bullish_diff + average_bearish_diff ) / 2

# stdev bullish and bearish candle OC
stdev_bullish_diff = bullish_rows['Open - Close'].std()
stdev_bearish_diff = bearish_rows['Open - Close'].std()
sigma = ( stdev_bullish_diff + stdev_bearish_diff ) / 2


# assign strenght bias
df['Strength'] = np.where(df['Open - Close'] >= mu + sigma, 'Strong',
                          np.where((df['Open - Close'] > mu-sigma) & (df['Open - Close'] < mu + sigma), 'Medium', 'Weak'))

df

,Timestamp,Bid,Close,Open,High,Low,Open - Close,Upper Wick,Lower Wick,Bias,Strength
1000,2023-12-01 00:21:58.140000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298,0.00073,0.00019,0.00007,Bullish,Medium
1001,2023-12-01 00:22:00.557000+00:00,1.26377,1.26377,1.26306,1.26397,1.26298,0.00071,0.00020,0.00008,Bullish,Medium
1002,2023-12-01 00:22:00.912000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298,0.00073,0.00019,0.00007,Bullish,Medium
1003,2023-12-01 00:22:01.166000+00:00,1.26378,1.26378,1.26306,1.26397,1.26298,0.00072,0.00019,0.00008,Bullish,Medium
1004,2023-12-01 00:22:01.317000+00:00,1.26378,1.26378,1.26304,1.26397,1.26298,0.00074,0.00019,0.00006,Bullish,Medium
...,...,...,...,...,...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795,0.00024,0.00022,0.00015,Bearish,Medium
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795,0.00025,0.00023,0.00013,Bearish,Medium
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795,0.00027,0.00021,0.00013,Bearish,Medium
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795,0.00035,0.00016,0.00010,Bearish,Medium
